In [1]:
import pandas as pd
import numpy as np
import os
from functools import reduce
import re
from scipy.io import savemat

In [31]:
def find_subject(subject_code='A001',path='LOG/'):
    """
    Read path, then search in given path person with
    given subject code'
    """
    files = [file for file in os.listdir('LOG/') if 'log' in file]
    return [file for file in files if subject_code in file]

def read_file(filename):
    """
    Read file, check whether it contains proper
    columns (based on first one) and return 
    DataFrame.
    """
    #read file, skiping rows with metadata
    file = pd.read_csv(filename,sep="\t",skiprows=2)
    #check columns (whether first one is subject)
    if np.any(file.columns.str.contains('Subject')):
        #delete Response (irrelevant for this study, but 
        #ther are some troubles with it)
        file = file.loc[file['Event Type'] != 'Response']
        #if columns are proper, restart index and return
        return file.reset_index()
    else:
        #if columns are not proper, read with skipping
        #different n of rows
        file = pd.read_csv(filename,"\t",skiprows=3)
        #drop unnecessary rows
        file = file.dropna(subset = ['Subject'])
        #delete Response (irrelevant for this study, but 
        #ther are some troubles with it)
        file = file.loc[file['Event Type'] != 'Response']
        #if columns are proper, restart index and return
        return file.reset_index()
    
def get_arg_block(data,blockname='block'):
    """Get arg of blocks in data"""
    return (data
            .loc[(data['Code']
                  .str.lower()
                  .str.contains(blockname.lower())),:]
            .index)

def time_correction(data):
    """
    Corrects time in data from Presentation. 
    Reduce all Time column by time value of first event, 
    then divides by 10000 (to turn into ms).
    Returns whole DataFrame.
    """
    data['Time'] = (data['Time'] - data['Time'].iloc[0])/10000
    return data

def create_file(filename,fixed_window=0):
    """
    Creates file with given filename
    """
    df = read_file(filename)
    df = time_correction(df)
    args = get_arg_block(df,r'S*P')
    df = df.loc[args]

    result = pd.DataFrame(columns=['Code','Onset','Offset'])
    for ind in range(len(df)):
        part = df.iloc[ind][['Code','Time']]
        try:
            if 'inter' in part['Code']:
                offset = part['Time'] + 15
                duration = 15
            else:
                offset = df.iloc[ind+1]['Time']
                duration = 13
            if fixed_duration>0:
                duration = fixed_duration
        except:
            pass

        result = pd.concat([result,
                            pd.DataFrame({'Code':part['Code'],
                             'Onset':part['Time'],
                             'Offset':offset,
                             'Duration':duration},index=[0])])
    return result.reset_index()


def create_file_end_recording(filename):
    """
    Creates file with given filename; 
    it is alternative way, in which 
    """
    df = read_file(filename)
    df = time_correction(df)
    args = get_arg_block(df,r'S*P')
    df = df.loc[args]

    result = pd.DataFrame(columns=['Code','Onset','Offset'])
    for ind in range(len(df)):
        part = df.iloc[ind][['Code','Time']]
        try:
            if 'end' in part['Code']:
                offset = part['Time'] + 15
                duration = 15
            else:
                offset = df.iloc[ind+1]['Time']
                duration = 13
        except:
            pass

        result = pd.concat([result,pd.DataFrame({'Code':part['Code'],
                      'Onset':part['Time'],
                      'Offset':offset,
                      'Duration':duration},index=[0])])
    return result.reset_index()

def generate_timing_conditions(data,prefix):
    for condition,name in zip(['S','SN'],
                              ['krytyka','neutralne']):
        for modality_sign,modality in zip(['$','inter'],
                                          ['s','i']):
            for part in ['1','2','3','123','4']:
                (data.loc[data['Code']
                      .str
                      .contains(fr'{condition}\d_P[{part}]{modality_sign}'),]
                 .to_csv(f'{prefix}_{name}_{modality}_P{part}.tsv'))
    if any(data['Code'].str.contains(fr'S\d_P\d_T')):
        for modality_sign,modality in zip(['$','inter'],
                                          ['s','i']):
            for part in ['1','2','3','123','4','5','6','7','4567']:
                (data.loc[data['Code']
                      .str
                      .contains(fr'S\d_P[{part}]_T{modality_sign}'),]
                 .to_csv(f'{prefix}_therapy_{modality}_P{part}_therapy.tsv'))
    elif any(data['Code'].str.contains(fr'SN\d_P\d_K')):
        for modality_sign,modality in zip(['$','inter'],
                                          ['s','i']):
            for part in ['1','2','3','123','4','5','6','7','4567']:
                (data.loc[data['Code']
                      .str
                      .contains(fr'SN\d_P[{part}]_K{modality_sign}'),]
                 .to_csv(f'{prefix}_therapy_{modality}_P{part}.tsv'))
                
def FSL_fix(data):
    """
    Return data, with columns necessary for analysis in FSL.
    """
    data['weight'] = np.ones(len(data))
    data.columns = ['index','trial_type','onset','offset','duration','weight']
    data['onset'] = np.round(data['onset'])
    return data.loc[:,['onset','duration','weight','trial_type']]

def generate_timing_conditions_FSL(data,prefix):
    data = FSL_fix(data)
    for condition,name in zip(['S','SN'],
                              ['krytyka','neutralne']):
        for modality_sign,modality in zip(['$','inter'],
                                          ['s','i']):
            for part in ['1','2','3','123','4']:
                (data.loc[data['trial_type']
                      .str
                      .contains(fr'{condition}\d_P[{part}]{modality_sign}'),['onset','duration','weight']]
                 .to_csv(f'{prefix}_{name}_{modality}_P{part}.tsv',
                         '\t',
                         index=False,
                         header=False))
    if any(data['trial_type'].str.contains(fr'S\d_P\d_T')):
        for modality_sign,modality in zip(['$','inter'],
                                          ['s','i']):
            for part in ['1','2','3','123','4','5','6','7','4567']:
                (data.loc[data['trial_type']
                      .str
                      .contains(fr'S\d_P[{part}]_T{modality_sign}'),['onset','duration','weight']]
                 .to_csv(f'{prefix}_therapy_{modality}_P{part}_therapy.tsv',
                         '\t',
                         index=False,
                         header=False))
    elif any(data['trial_type'].str.contains(fr'SN\d_P\d_K')):
        for modality_sign,modality in zip(['$','inter'],
                                          ['s','i']):
            for part in ['1','2','3','123','4','5','6','7','4567']:
                (data.loc[data['trial_type']
                      .str
                      .contains(fr'SN\d_P[{part}]_K{modality_sign}'),['onset','duration','weight']]
                 .to_csv(f'{prefix}_therapy_{modality}_P{part}.tsv',
                         '\t',
                         index=False,
                         header=False))
                

def generate_timing_conditions_SPM(data,prefix):
    data = FSL_fix(data)
    result = {'names':[],
              'onsets':[],
              'durations':[]}
    for condition,name in zip(['S','SN'],
                              ['krytyka','neutralne']):
        for modality_sign,modality in zip(['$','inter'],
                                          ['s','i']):
            for part in ['1','2','3','4']:
                result['names'].append(fr"{name}_{modality}_P{part}")
                result['onsets'].append((data.loc[data['trial_type']
                                                  .str
                                                  .contains(fr'{condition}\d_P[{part}]{modality_sign}'),
                                                  ['onset']])
                                        .values)
                result['durations'].append((data.loc[data['trial_type']
                      .str
                      .contains(fr'{condition}\d_P[{part}]{modality_sign}'),['duration']]).values)

    if any(data['trial_type'].str.contains(fr'S\d_P\d_T')):
        for modality_sign,modality in zip(['$','inter'],
                                          ['s','i']):
            for part in ['1','2','3','123','4','5','6','7']:
                result['names'].append(f'therapy_{modality}_P{part}')
                result['onsets'].append((data.loc[data['trial_type']
                      .str
                      .contains(fr'S\d_P[{part}]_T{modality_sign}'),['onset']]).values)
                result['durations'].append((data.loc[data['trial_type']
                      .str
                      .contains(fr'S\d_P[{part}]_T{modality_sign}'),['duration']]).values)

    elif any(data['trial_type'].str.contains(fr'SN\d_P\d_K')):
        for modality_sign,modality in zip(['$','inter'],
                                          ['s','i']):
            for part in ['1','2','3','4','5','6','7']:
                result['names'].append(f'therapy_{modality}_P{part}')
                result['onsets'].append((data.loc[data['trial_type']
                      .str
                      .contains(fr'SN\d_P[{part}]_K{modality_sign}'),['onset']]).values)
                result['durations'].append((data.loc[data['trial_type']
                      .str
                      .contains(fr'SN\d_P[{part}]_K{modality_sign}'),['duration']]).values)
    tmp=np.zeros(len(result['names']),
                 dtype=object)
    tmp[:]=result['names']
    result['names']  = tmp
    savemat(f'{prefix}.mat',uno)
    return result


In [43]:
file = read_file('LOG/JM2022_B004_TP1-Run2_functionalScan.log')
file = create_file('LOG/JM2022_B004_TP1-Run2_functionalScan.log')
generate_timing_conditions_SPM(file,'test')

In [42]:
path='LOG/'
for raw_file in os.listdir('LOG/'):
    #check whether file contains subject's code
    if len(re.findall(r'b\d+',raw_file.lower()))>0:
        #if it contains code, try create directory with subject code,
        #if this directory exists, pass this action.
        subject_code = re.findall(r'b\d+',raw_file.lower())[0].upper()
        try:
            os.mkdir(subject_code)
            os.mkdir(f'{subject_code}\ses-1')
            os.mkdir(f'{subject_code}\ses-2')
        except FileExistsError:
            pass
        #
        files = find_subject(subject_code,path)
        for filename in files:
            print(filename)
            try:
                result = create_file(f"{path}{filename}")
                run = re.findall(r'Run\d',filename)[0]
                if 'tp1' in filename.lower():
                    generate_timing_conditions_FSL(result,f"{subject_code}\ses-1\\{subject_code}{run}")
                elif 'tp5' in filename.lower():
                    generate_timing_conditions_FSL(result,f"{subject_code}\ses-2\\{subject_code}{run}")
            except Exception as e:
                print(f"Problem with {filename}\n{e}")

In [41]:
path='LOG/'
output_path = 'output/'
for raw_file in os.listdir('LOG/'):
    #check whether file contains subject's code
    if len(re.findall(r'b\d+',raw_file.lower()))>0:
        #if it contains code, try create directory with subject code,
        #if this directory exists, pass this action.
        subject_code = re.findall(r'b\d+',raw_file.lower())[0].upper()
        try:
            os.mkdir(f"{output_path}sub-{subject_code}")
            os.mkdir(f'{output_path}sub-{subject_code}\ses-1')
            os.mkdir(f'{output_path}sub-{subject_code}\ses-2')
        except FileExistsError:
            pass
        #
        files = find_subject(subject_code,path)
        for filename in files:
            print(filename)
            try:
                result = create_file(f"{path}{filename}")
                run = re.findall(r'Run\d',filename)[0]
                if 'tp1' in filename.lower():
                    generate_timing_conditions_SPM(result,f"{output_path}sub-{subject_code}\ses-1\\sub-{subject_code}_ses-1_{run}")
                elif 'tp5' in filename.lower():
                    generate_timing_conditions_SPM(result,f"{output_path}sub-{subject_code}\ses-2\\sub-{subject_code}_ses-2_{run}")
            except Exception as e:
                print(f"Problem with {filename}\n{e}")